### Activate PTS

In [1]:
from pts.do.prompt import do
#%matplotlib inline
#%matplotlib notebook
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import logging # This is a Python standard package.
import pts.simulation as sm
import pts.utils as ut
import pts.visual as vis
import pts.do
import numpy as np
#pts.do.initializePTS()

In [2]:
num=4
repo = f"/home/hoseung/Work/data/NH/mock_recipe_test/runs/test{num}/"
fname = f"try{num}.ski"

## Detecting 'important' peaks and overall shape

In [3]:
from pts import band

In [68]:
thres = 0.05
for bname in band.builtinBandNames():
    bd = band.broadband.BroadBand(bname)
    xx, tt = bd.transmissionCurve()
    filtered = savgol_filter(tt, min((17, 2*int(np.floor(len(xx)/2))-3)), 3)
    frac = (filtered - tt.value)/tt.value.ptp()
    ind = np.where(frac > thres)[0]
    print("# peaks =", len(ind))
    extreme = np.where(local_extrema(filtered))[0]
    print("There're {:d} extrema".format(len(extreme)))
    plt.plot(xx,tt)
    plt.plot(xx,filtered)
    plt.scatter(xx[ind], tt[ind])
    plt.savefig(bname+".png")
    plt.close()


# peaks = 9
There're 3 extrema
# peaks = 18
There're 10 extrema
# peaks = 5
There're 4 extrema
# peaks = 0
There're 5 extrema
# peaks = 0
There're 5 extrema
# peaks = 1
There're 2 extrema
# peaks = 3
There're 2 extrema
# peaks = 11
There're 3 extrema
# peaks = 8
There're 2 extrema
# peaks = 4
There're 2 extrema
# peaks = 3
There're 2 extrema
# peaks = 5
There're 2 extrema
# peaks = 0
There're 15 extrema
# peaks = 0
There're 6 extrema
# peaks = 0
There're 5 extrema
# peaks = 1
There're 9 extrema
# peaks = 0
There're 6 extrema
# peaks = 7
There're 6 extrema
# peaks = 2
There're 2 extrema
# peaks = 7
There're 2 extrema
# peaks = 6
There're 3 extrema
# peaks = 4
There're 2 extrema
# peaks = 3
There're 3 extrema
# peaks = 4
There're 7 extrema
# peaks = 1
There're 3 extrema
# peaks = 3
There're 2 extrema
# peaks = 2
There're 4 extrema
# peaks = 0
There're 12 extrema
# peaks = 0
There're 12 extrema
# peaks = 0
There're 8 extrema
# peaks = 0
There're 13 extrema
# peaks = 0
There're 8 extrema
#

In [10]:
from scipy.signal import savgol_filter

In [15]:
filtered = savgol_filter(tt, 7, 2)

In [23]:
def local_extrema(a):
    return np.r_[True, a[1:] < a[:-1]] & np.r_[a[:-1] < a[1:], True]

In [16]:
plt.plot(xx, tt)
plt.plot(xx, filtered)
plt.show()

In [29]:
skifile = sm.SkiFile(repo+fname)

In [14]:
# Modifying ski file
skifile = sm.SkiFile(repo+fname)
skifile.setNumPrimaryPackets(1e9)
fname = repo+"HQ_testing.ski"
skifile.saveTo(fname)
logging.info("Saved {}".format(fname))

   Saved /home/hoseung/Work/data/NH/mock_recipe_test/runs/test1/HQ_testing.ski
15/04/2020 17:29:23.478   Saved /home/hoseung/Work/data/NH/mock_recipe_test/runs/test1/HQ_testing.ski


## Runing SKIRT from Python -- (but doesn't work in this setting..)

In [19]:
# Performing SKIRT simulations
logging.info("Executing {}".format(repo+fname))
skirt = sm.Skirt()
simulation = skirt.execute(repo+fname, console='brief')

   Executing /home/hoseung/Work/data/NH/mock_recipe_test/runs/test1/try1.ski
15/04/2020 18:31:44.079   Executing /home/hoseung/Work/data/NH/mock_recipe_test/runs/test1/try1.ski


15/04/2020 18:31:44.196   Welcome to SKIRT v9.0 (git d9c0c65 built on 13/02/2020 at 22:08:41)
15/04/2020 18:31:44.197   Running on hoseung-P75-Creator-9SD for hoseung
15/04/2020 18:31:44.197   Constructing a simulation from ski file '/home/hoseung/Work/data/NH/mock_recipe_test/runs/test1/try1.ski'...
15/04/2020 18:31:44.200 * *** Error: Error in XML stream '/home/hoseung/Work/data/NH/mock_recipe_test/runs/test1/try1.ski' at line 9
15/04/2020 18:31:44.200 * *** Error: Item of type 'MonteCarloSimulation' has no property named 'cosmology'
15/04/2020 18:31:44.200 * *** Error: On line 185 in file /home/hoseung/Work/SKIRT/git/SMILE/fundamentals/XmlReader.cpp
15/04/2020 18:31:44.200 * *** Error: In function throwError
15/04/2020 18:31:44.200 * *** Error: Call stack:
15/04/2020 18:31:44.200 * *** Error: /home/hoseung/Work/SKIRT/release/SKIRT/main/skirt(_ZN10FatalErrorC2ENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEPKciS7_+0x6fa) [0x564ab61ed2da]
15/04/2020 18:31:44.200 * *** Error: /hom

## Starting from an already performed simulation result (not run in this Python instance)

In [11]:
sim = sm.createSimulation(outDirPath=repo, prefix='try4')
skifile = sim.parameters()

In [40]:
# SED plot
micron = sm.unit("micron")
vis.plotSeds(sim, decades=4, figSize=(7,5), outFileName="try1_sed.png")

   Created /home/hoseung/Work/data/NH/mock_recipe_test/runs/test4/try1_sed.png
15/04/2020 19:40:41.877   Created /home/hoseung/Work/data/NH/mock_recipe_test/runs/test4/try1_sed.png
15/04/2020 19:40:41.877   Created /home/hoseung/Work/data/NH/mock_recipe_test/runs/test4/try1_sed.png


In [45]:
# Loading FITS files
# Get the absolute path to the output file based on basic Sim info from sim instance.
# indexing [0] for the first of possibly multiple intruments.
totalfluxpath = sim.instruments()[0].outFilePaths(fileType="total.fits")[0]
# loadFits & getFitsAxes return 'astropy' quantities with units attached.
datacube = sm.loadFits(totalfluxpath)
x,y,wavelengths = sm.getFitsAxes(totalfluxpath)

In [55]:
frame = datacube[:,:,20]
wavelength = wavelengths[20]
logging.info("Observed frame has {} by {} pixels".format(*frame.shape))
logging.info("Maximum surface brightness at {:.5f} is {:.5e}".format(wavelength, frame.max()))

   Observed frame has 1284 by 1284 pixels
15/04/2020 19:51:30.837   Observed frame has 1284 by 1284 pixels
15/04/2020 19:51:30.837   Observed frame has 1284 by 1284 pixels
   Maximum surface brightness at 0.28182 micron is 8.70836e-13 W / (arcsec2 m2 micron)
15/04/2020 19:51:30.840   Maximum surface brightness at 0.28182 micron is 8.70836e-13 W / (arcsec2 m2 micron)
15/04/2020 19:51:30.840   Maximum surface brightness at 0.28182 micron is 8.70836e-13 W / (arcsec2 m2 micron)


### reading probe outputs

In [ ]:
for probe in simulation.probes():
    if "CellProperties" in probe.type():
        filepath = probe.outFilePaths()[0]
        volume, density = sm.loadColumns(filepath, "volume,dust mass density")
        mass = volume*density
    if "DustTemperaturePerCell" in probe.type():
        filepath = probe.outFilePaths()[0]
        temperature, = sm.loadColumns(filepath, "dust temperature")
Tavg = (mass*temperature).sum() / mass.sum()

## RGB image

Looking at pts/visual/do/make_images.py  
Two use cases:
1. Specify wavelength  
2. Specify bands  

Q: Any support for SB limit??


In [146]:
sim._parameters.instrumentNames()

['maraston_zubkoHQ']

In [142]:
import shutil

In [148]:
sim.skiFilePath()

In [171]:
for num in range(22, 49):
    repo = f"/home/hoseung/Work/data/NH/mock_recipe_test/runs/test{num}/"
    #fname = f"try{num}_LQ.ski"
    try:
        sim = sm.createSimulation(outDirPath=repo, prefix=f'try{num}')
        skifile = sim.parameters()
    except:
        print(f"No outputs in {num}")
        continue
    
    try:
        totalfluxpath = sim.instruments()[0].outFilePaths(fileType="total.fits")[0]
    except:
        print("No output?", num)        
    old_name = totalfluxpath.__str__()
    name_base = old_name.replace("HQ", "")
    shutil.move(old_name, name_base.replace("total", "real_HQ_total"))
    shutil.move(name_base, old_name)

    try:
        sim = sm.createSimulation(outDirPath=repo, prefix=f'try{num}')
        skifile = sim.parameters()
    except:
        print(f"No outputs in {num}")
        continue
    
    try:
        totalfluxpath = sim.instruments()[0].outFilePaths(fileType="total.fits")[0]
    except:
        print("No output?", num)
        
    print(totalfluxpath)
    cube = sm.loadFits(totalfluxpath)# replace to force to load LQ output.
    if cube.max() == 0:
        print("Empty fits", num)
        continue

    import pts.band as bnd
    wavelengths = None

    if wavelengths is not None:
        tuples = { name: wavelengths << sm.unit("micron") }
        for sim in sims:
            vis.makeRGBImages(sim, wavelengthTuples=tuples, fileType=type)

    # try bands
    colors = "SDSS_Z,SDSS_R,SDSS_U"#,2MASS_2MASS_H,2MASS_2MASS_J,2MASS_2MASS_KS"
    name = "SDSS_RGB_LQ"
    output_type = "total"
    segments = colors.split(',')
    if len(segments) != 3:
        raise ut.UserError("colors argument must have three comma-separated segments")

    try: bands = [ bnd.BroadBand(segment) for segment in segments ]
    except ValueError: bands = None
    if bands is not None:
        contributions = [ (bands[0], 1, 0, 0), (bands[1], 0, 1, 0), (bands[1], 0, 0, 1) ]
        # Could loop over sims
        vis.makeConvolvedRGBImages(sim, contributions=contributions, fileType=output_type, name=name,
                                  stretch=np.arcsinh, decades=5)
        vis.makeConvolvedRGBImages(sim, contributions=contributions, fileType=output_type, name=name,
                                  stretch='log', decades=5)    
        
    

/home/hoseung/Work/data/NH/mock_recipe_test/runs/test22/try22_blackbody_wein_LMCHQ_total.fits
Empty fits 22
No output? 23


FileNotFoundError: [Errno 2] No such file or directory: '/home/hoseung/Work/data/NH/mock_recipe_test/runs/test22/try22_blackbody_wein_LMC_total.fits'

## .log file 지우면 안 됨.  한 directory에는 하나의 run만. 

In [110]:
sim = sm.Simulation(skiFilePath=repo+fname)

In [118]:
totalfluxpath.__str__()

'/home/hoseung/Work/data/NH/mock_recipe_test/runs/test4/try4_maraston_zubkoHQ_total.fits'

In [65]:
totalfluxpath

PosixPath('/home/hoseung/Work/data/NH/mock_recipe_test/runs/test4/try4_maraston_zubkoHQ_total.fits')

In [74]:
bnd.builtinBandNames()

dict_keys(['2MASS_2MASS_H', '2MASS_2MASS_J', '2MASS_2MASS_KS', 'GALEX_GALEX_FUV', 'GALEX_GALEX_NUV', 'GENERIC_JOHNSON_B', 'GENERIC_JOHNSON_I', 'GENERIC_JOHNSON_J', 'GENERIC_JOHNSON_M', 'GENERIC_JOHNSON_R', 'GENERIC_JOHNSON_U', 'GENERIC_JOHNSON_V', 'HERSCHEL_PACS_100', 'HERSCHEL_PACS_160', 'HERSCHEL_PACS_70', 'HERSCHEL_SPIRE_250', 'HERSCHEL_SPIRE_350', 'HERSCHEL_SPIRE_500', 'IRAS_IRAS_100', 'IRAS_IRAS_12', 'IRAS_IRAS_25', 'IRAS_IRAS_60', 'SLOAN_SDSS_G', 'SLOAN_SDSS_I', 'SLOAN_SDSS_R', 'SLOAN_SDSS_U', 'SLOAN_SDSS_Z', 'SPITZER_IRAC_I1', 'SPITZER_IRAC_I2', 'SPITZER_IRAC_I3', 'SPITZER_IRAC_I4', 'SPITZER_MIPS_160', 'SPITZER_MIPS_24', 'SPITZER_MIPS_70', 'SWIFT_UVOT_B', 'SWIFT_UVOT_UVM2', 'SWIFT_UVOT_UVW1', 'SWIFT_UVOT_UVW2', 'SWIFT_UVOT_U', 'SWIFT_UVOT_V', 'TNG_NICS_H', 'TNG_NICS_J', 'TNG_NICS_K', 'TNG_OIG_B', 'TNG_OIG_R', 'TNG_OIG_U', 'TNG_OIG_V', 'UKIRT_UKIDSS_H', 'UKIRT_UKIDSS_J', 'UKIRT_UKIDSS_K', 'UKIRT_UKIDSS_Y', 'UKIRT_UKIDSS_Z', 'WISE_WISE_W1', 'WISE_WISE_W2', 'WISE_WISE_W3', 'WISE_WI

In [6]:
np.add.__name__

'add'